In [ ]:
import requests
from bs4 import BeautifulSoup
from googlesearch import search
from tqdm import tqdm

# Running Local deepseek 14b model. Specs, intel i9-14900k, Nvidia RTX 4080, 64 gb DDR 5600 MHZ RAM
def call_ollama(prompt, model='deepseek-r1:14b'):
    response = requests.post(
        'http://localhost:11434/api/generate',
        json={
            'model': model,
            'prompt': prompt,
            'stream': False
        }
    )
    return response.json()['response']

# Deepseek to reccommend search query
def interpret_user_query(user_input):
    prompt = f"""
    You are a smart assistant helping with product discovery, specifically for prototyping intial products for a startup. You have knowledge in the field in which the
    user asks and knowledgable of the current landscape of the product. This is to improve the act of gathering necessary supplies for prototyping a viable product 
    based on user inputted conditions. The user said:
    '{user_input}'
    Convert this into a specific product search query and list key product attributes to look for. Don't add ther URLS here but these should be things to get exact products when inputted into
    a search engine to provided the user with their desired product. The list must adhere to this format: number . thing to search 
    1. search1
    2. search2
    3. search3
    4. search4
    5. search5
    This list should only contain the URLS to search for. Look for pricing and criteria factors as well. Provide brief reasoning for these generated entries to search for. 
    """
    return call_ollama(prompt)

# model feedback to make a decision
def provide_feedback(feedback_input, urls, summary):
    prompt = f"""
    As a smart assistant helping with shopping for supplies for prototyping a viable product for startups. The user said:
    '{feedback_input}' which was based on the urls here: {urls} which in turn was oringally created off ofthis query response: {summary}. 
    Make reccomendations based on this feedback to find a good product 
    """

    return call_ollama(prompt)
import re
def extract_entries(text):
    pattern = r"^\d+\.\s*(.+)$"
    entries = re.findall(pattern, text, re.MULTILINE)  #re.MULTILINE to handle multiple lines
    return entries

# Google
def get_search_results(query, num_results=5):
    return list(search(query, num_results=num_results))

# Web Scraper
def scrape_product_data(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        summary = ' '.join(p.get_text(strip=True) for p in paragraphs[:10])
        price_tags = soup.find_all(
            lambda tag: tag.name in ['span', 'div'] and any('price' in x.lower() for x in tag.get('class', []) + tag.get('id', '').split())
        )
        prices = [tag.get_text(strip=True) for tag in price_tags if tag.get_text(strip=True)]

        return {
            "summary": summary,
            "prices": prices
        }

    except Exception as e:
        return {"error": f"Error scraping {url}: {str(e)}"}

# -Summarizing web results
def summarize_scraped_data(scraped_data, original_query, products):
    prompt = f"""
    Given the following product info:
    {scraped_data}
    Extract the product's purpose, key specs, pros and cons in bullet points. Compare this to other products and give reasons for why each product is best for its use case. 
    The oriingal query that contains the criterai and impetus for the product being searched is here: {original_query}. The list of products urls that we are researching is {products}. Now
    there will be repeat products but likley from different vendors. In this case, examine the pricing differences and shipping times, and customer reviews for each vendor. Ensure that if repeats are found
    we only compare the vendors and only find unique data that is not found within other entries. There are price fields returned under price: , check to see that these numbers are consistent and
    display them if the remain consistent with the actual pricing of the product. 
    """
    return call_ollama(prompt)



In [2]:
user_input = True
user_input = input("Describe what you're looking for: ")
f = open("report.txt", "x")
f = open("report.txt", "a")
while (user_input):

    print("\n[1] Interpreting query...")
    f.write("\n[1] Interpreting query...")
    interpreted = str(interpret_user_query(user_input))
    filtered = (interpreted[interpreted.index('</think>') + 8:].rstrip()).strip()
    print(filtered)
    f.write(filtered)
    print("\n[2] Researching Reccommended Products")
    f.write("\n[2] Researching Reccommended Products")
    search_query_list = extract_entries(filtered) 
    global_url = 0
    all_urls = []
    for search_query in search_query_list:
        urls = get_search_results(search_query)
        all_urls.extend(urls)
        for i, url in enumerate(urls):
            print(f"[{global_url+i+1}] {url}")
            f.write(f"[{global_url+i+1}] {url}")
        global_url += 5
    print("\n[3] Scraping and summarizing...")
    f.write("\n[3] Scraping and summarizing...")
    for i in tqdm(range(len(all_urls))):
        raw_data = scrape_product_data(all_urls[i])
        summary = summarize_scraped_data(raw_data, user_input, all_urls)
        summary = str(interpret_user_query(summary))
        filtered_summary = (summary[summary.index('</think>') + 8:].rstrip()).strip()
        print(f"\nSummary from {all_urls[i]}:\n{filtered_summary}\n")
        f.write(f"\nSummary from {all_urls[i]}:\n{filtered_summary}\n")
    user_prompt = input("Are you satisified with this analysis? : Yes / No")
    if (user_prompt == 'Yes'):
        user_input = False
    else:
        user_feeedback = input("What changes should be made?")
        feedback = provide_feedback(user_feeedback, all_urls, user_input)
        print (feedback)
        f.write(feedback)
        user_input = False
    f.close()

    

        



[1] Interpreting query...
1. Stainless Steel Precision Screws
2. Brass Machine Screws
3. Fine Thread Screws for Small Scale Models
4. Coarse Thread Machine Screws
5. McMaster-Carr Tiny Screws for Architecture Models

[2] Researching Reccommended Products
[1] https://www.fastenersuperstore.com/products/812220/precision-shoulder-screws?srsltid=AfmBOooATMp_UEJ_gYsFC4hCzinppmcZhJE0oOOpp0au1cDo2LaddM8t
[2] https://www.aftfasteners.com/precision-shoulder-screws/
[3] https://accu-components.com/us/precision-slotted-shoulder-screws/50458-SFHZ-M5-6-A2
[4] https://www.mcmaster.com/products/precision-bolts/
[5] https://www.wmberg.com/products/fasteners-and-hardware/screws
[6] https://www.fastenere.com/14-20-slotted-round-head-machine-screws-solid-brass?srsltid=AfmBOorscHHs9U4gNL1qF7kXsx8bkBfiXmR4gyEREilGcFr0yeFsXEK8
[7] https://www.aspenfasteners.com/shop-by-material/brass/?srsltid=AfmBOooChp_85ebcBfNJLdIgURO908Bl9tgy4bIa0veVQjRzC-tNY3yv
[8] https://www.google.com/search?num=7
[9] https://www.mc

  4%|▍         | 1/25 [00:33<13:31, 33.81s/it]


Summary from https://www.fastenersuperstore.com/products/812220/precision-shoulder-screws?srsltid=AfmBOooATMp_UEJ_gYsFC4hCzinppmcZhJE0oOOpp0au1cDo2LaddM8t:
1. 1/8 inch diameter precision screws with 0.156 thread length  
2. High-quality 1/8 inch screw for scale models, fast shipping  
3. Precision 1/8 inch diameter screw with non-lead material  
4. McMaster-Carr 1/8 inch diameter screw, quick delivery  
5. Stainless steel 1/8 inch screw with 0.156 thread length  

**Reasoning**: Each search query targets specific attributes like diameter, thread length, material, and vendor reliability to find the most suitable screws for architectural models while considering practicality factors like shipping speed and safety standards.



  8%|▊         | 2/25 [01:04<12:16, 32.01s/it]


Summary from https://www.aftfasteners.com/precision-shoulder-screws/:
1. precision stainless steel shoulder screws  
2. slotted vs socket drive shoulder screws comparison  
3. scale model building fasteners stainless steel #4-#10  
4. reliable suppliers for miniature machine screws  
5. bulk pricing on precision shoulder screws  

**Reasoning:**
- **1. precision stainless steel shoulder screws:** Directly targets the material and type needed, ensuring results are specific to high-quality options.
- **2. slotted vs socket drive shoulder screws comparison:** Helps determine which drive type suits their application best by comparing the two common types.
- **3. scale model building fasteners stainless steel #4-#10:** Tailored for their niche, focusing on sizes suitable for precise scale models.
- **4. reliable suppliers for miniature machine screws:** Ensures they find trustworthy vendors with small screw options, crucial for detailed models.
- **5. bulk pricing on precision shoulder scr

...


Summary from https://www.mcmaster.com/products/miniature-bolts/:
1. precision stainless steel screws M5 socket cap head  
   Reasoning: Specifies material (stainless steel), size (M5), and head type (socket cap) for industrial-grade precision.

2. high-quality brass machine screws M6 fine thread  
   Reasoning: Offers a cost-effective alternative to stainless steel while specifying thread type (fine) and size (M6).

3. miniature steel screws M5 slotted head for precise assembly  
   Reasoning: Targets small-scale projects with specific focus on size (M5) and head type (slotted).

4. industrial-grade titanium screws M6 Phillips head  
   Reasoning: Provides a premium option with high strength-to-weight ratio, specifying size and head type.

5. budget-friendly steel machine screws M5 coarse thread  
   Reasoning: Focuses on affordability while maintaining necessary durability and thread type (coarse).



 96%|█████████▌| 24/25 [15:20<00:40, 40.53s/it]


Summary from https://www.reddit.com/r/AskEngineers/comments/132yx4j/mcmastercarr_bulkproduction_machine_screw/:
1. precision machine screws McMaster-Carr  
   - McMaster-Carr offers a wide range of precision machine screws in various sizes and materials, making it ideal for small-scale production or prototyping.

2. bulk machine screws Bolt Depot  
   - Bolt Depot specializes in bulk pricing for larger orders, which is perfect for production needs.

3. miniature machine screws USMicroScrew  
   - USMicroScrew focuses on miniature screws, making it a great option for applications like architectural scale models.

4. stainless steel machine screws McMaster-Carr  
   - Stainless steel machine screws from McMaster-Carr are durable and resistant to corrosion, ideal for long-lasting use.

5. fasteners stainless steel Phillips head Amazon  
   - Amazon offers a variety of fasteners with quick shipping options, including stainless steel Phillips head screws that are both affordable and reliab

100%|██████████| 25/25 [15:59<00:00, 38.38s/it]



Summary from https://www.mcmaster.com/:
1. precision machine screws M5x6  
2. miniature stainless steel bolts 14-20 thread  
3. Accu-components slotted shoulder screw model 50458-SFHZ-M5-6-A2  
4. McMaster-Carr precision machine screws for scale models  
5. brass pan head screws for architectural scale models

<think>
Okay, so I'm trying to help someone who's working on a scale replica for an architecture startup. They mentioned they need well-machined screws that fit exact dimensions and are reliable. Let me think through how to approach this.

First, understanding the use case: it's a scale model of buildings, which means precision is key. The screws need to be small but sturdy enough to hold the structure without being too bulky. They probably need consistency in size and thread so that they fit properly into their models.

Looking at the URLs provided, I see references to various suppliers like McMaster-Carr, Bolt Depot, Aspen Fasteners, etc. From what I know, McMaster-Carr is a r

In [19]:
print (all_urls)

['https://randomnerdtutorials.com/esp32-lora-rfm95-transceiver-arduino-ide/', 'https://www.aliexpress.com/item/10000290160030.html', 'https://heltec.org/project/wifi-lora-32-v3/', 'https://www.aliexpress.com/i/32957042456.html', 'https://circuitdigest.com/microcontroller-projects/interfacing-sx1278-lora-module-with-esp32', 'https://www.st.com/en/evaluation-tools/32l496gdiscovery.html', 'https://www.digikey.com/en/products/detail/stmicroelectronics/STM32L496G-DISCO/6697038', 'https://www.st.com/content/st_com/en/products/evaluation-tools/product-evaluation-tools/mcu-eval-tools/stm32-mcu-eval-tools/stm32-mcu-discovery-kits/32l496gdiscovery.html', 'https://www.mouser.com/ProductDetail/STMicroelectronics/STM32L496G-DISCO?qs=GFgPWxtVJSbWvqPi%252BdpPVw%3D%3D&srsltid=AfmBOorZEHF9AinTbkgvskc1CaA636F7th7WJoWoVIoDXYqaMKpvNiSt', 'https://docs.zephyrproject.org/latest/boards/st/stm32l496g_disco/doc/index.html', 'https://www.nordicsemi.com/Products/nRF52833', 'https://www.nordicsemi.com/Products/De